In [563]:
#Starter code provided by instructor
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [564]:
#Setting variables to get sum, average, and %s 
Total_Schools = school_data_complete['School ID'].nunique()

Total_Students = (school_data_complete['Student ID'].nunique())

Total_Budget = school_data['budget'].sum()

Average_Math_Score = school_data_complete['math_score'].mean()

Average_Reading_Score = school_data_complete['reading_score'].mean()

Students_Passed_Math = school_data_complete.loc[school_data_complete['math_score'] >= 70]

Passing_Math_Percentage = (Students_Passed_Math['Student ID'].nunique() / Total_Students) *100

Students_Passed_Reading = school_data_complete.loc[school_data_complete['reading_score'] >= 70]

Passing_Reading_Percentage = (Students_Passed_Reading['Student ID'].nunique() / Total_Students) *100

Overall_Passing_Students = school_data_complete[(school_data_complete['reading_score'] >= 70) &
                                                  (school_data_complete['math_score'] >= 70)].nunique()

Overall_Passing_Students = (Overall_Passing_Students['Student ID'] / Total_Students) *100

In [565]:
#Creating the District df
District_Summary_df = pd.DataFrame({'Total Schools': [Total_Schools],
                                  'Total Students': [Total_Students],
                                  'Total Budget': [Total_Budget],
                                  'Average Math Score': [Average_Math_Score],
                                  'Average Reading Score': [Average_Reading_Score],
                                  '% Passing Math': [Passing_Math_Percentage],
                                  '% Passing Reading': [Passing_Reading_Percentage],
                                  '% Overall Passing': [Overall_Passing_Students]})

In [566]:
#Formating Total Budget and Total Students
District_Summary_df['Total Budget'] = District_Summary_df['Total Budget'].map('${:,.2f}'.format)
District_Summary_df['Total Students'] = District_Summary_df['Total Students'].map('{:,.0f}'.format)

In [567]:
District_Summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [568]:
school_data.set_index('school_name');

In [569]:
#Creating the School df and renaming columns
School_Group_df = school_data[['school_name', 'type', 'size', 'budget']].set_index('school_name')
School_Group_df = School_Group_df.rename(columns={'type': 'School Type',
                                                  'size': 'Total Students',
                                                  'budget': 'Total School Budget'})

In [570]:
#Adding a column for Budget per Student
School_Group_df['Per Student Budget'] = School_Group_df['Total School Budget']/School_Group_df['Total Students']

In [572]:
#Grouping school data by 'shool_name'
School_Group_Info_df = school_data_complete.groupby(['school_name'])

In [573]:
#Passing math, reading, and both totals by school
Pas_Math_School = school_data_complete[school_data_complete['math_score']
                                       >= 70].groupby('school_name')['School ID'].count()
Pas_Reading_School = school_data_complete[school_data_complete['reading_score']
                                          >= 70].groupby('school_name')['School ID'].count()
Pas_Both_School = school_data_complete[(school_data_complete['math_score']
                                       >= 70) & (school_data_complete['reading_score']
                                               >= 70)].groupby('school_name')['School ID'].count()

In [577]:
#Merging the series into df
Perc_df = pd.merge(Pas_Math_School, Pas_Reading_School, on="school_name")

In [578]:
Perc_df = pd.merge(Perc_df, Pas_Both_School, on="school_name")

In [580]:
Perc_df = Perc_df.rename(columns={'School ID_x': 'Passed Math Students',
                                                  'School ID_y': 'Passed Reading Students',
                                                  'School ID': 'Passed Both'})

In [581]:
Perc_df = pd.merge(Perc_df, School_Group_df['Total Students'], on="school_name")

In [582]:
#Passing math, reading, and both %s by school
Perc_df['% Passing Math'] = (Perc_df['Passed Math Students']/School_Group_df['Total Students'])*100
Perc_df['% Passing Reading'] = (Perc_df['Passed Reading Students']/School_Group_df['Total Students'])*100
Perc_df['% Overall Passing'] = (Perc_df['Passed Both']/School_Group_df['Total Students'])*100

In [584]:
#Creating a final percentage df
Perc_df_Final = Perc_df[['% Passing Math', '% Passing Reading', '% Overall Passing']]

In [586]:
#Creating a school summary df with groupby 'school_name' df
School_Summary_df2 = School_Group_Info_df[['school_name', 'math_score', 'reading_score']]

In [588]:
#getting the average math and reading score by school
School_Summary_df3 = pd.DataFrame(School_Summary_df2.mean().reset_index("school_name"))

In [590]:
#Resetting the index and renaming columns
School_Summary_df3.head().set_index('school_name')

School_Summary_df3 = School_Summary_df3.rename(columns={'reading_score': 'Average Reading Score',
                                                        'math_score': 'Average Math Score'})

In [591]:
#Merging the the school group, school summary, and percentage df
merge_df = pd.merge(School_Group_df, School_Summary_df3, on="school_name")
merge_df = pd.merge(merge_df, Perc_df_Final, on="school_name")

In [592]:
#Formating Total School Budget and Per Student Budget
merge_df['Total School Budget'] = merge_df['Total School Budget'].map("${:,.2f}".format)
merge_df['Per Student Budget'] = merge_df['Per Student Budget'].map("${:,.2f}".format)

In [593]:
#Resetting the index and renaming columns for final School Summary df
merge_df = merge_df.rename(columns={'school_name': 'School Name'})
merge_df = merge_df.set_index('School Name')

In [594]:
#sorting index
merge_df.sort_index(axis = 0)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [596]:
#Sorting data high to low on '% Overall Passing' and displaying top 5
Top_Performing_Schools_df = merge_df.sort_values(by=['% Overall Passing'], ascending=False)
Top_Performing_Schools_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [608]:
#Sorting data low to high on '% Overall Passing' and displaying bottom 5
Bottom_Performing_Schools_df = merge_df.sort_values(by=['% Overall Passing'], ascending=True)
Bottom_Performing_Schools_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [613]:
#Creating series for average math score by grade and school
ninth_grade_math = school_data_complete.loc[school_data_complete['grade'] == '9th'].groupby('school_name')['math_score'].mean()
tenth_grade_math = school_data_complete.loc[school_data_complete['grade'] == '10th'].groupby('school_name')['math_score'].mean()
eleventh_grade_math = school_data_complete.loc[school_data_complete['grade'] == '11th'].groupby('school_name')['math_score'].mean()
twelfth_grade_math = school_data_complete.loc[school_data_complete['grade'] == '12th'].groupby('school_name')['math_score'].mean()

In [619]:
#Creating df for math scores by grade and school
math_by_grade = pd.DataFrame({'9th': ninth_grade_math,
                              '10th': tenth_grade_math,
                              '11th': eleventh_grade_math,
                              '12th': twelfth_grade_math})

In [620]:
math_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [621]:
#Creating series for average reading score by grade and school
ninth_grade_reading = school_data_complete.loc[school_data_complete['grade'] == '9th'].groupby('school_name')['reading_score'].mean()
tenth_grade_reading = school_data_complete.loc[school_data_complete['grade'] == '10th'].groupby('school_name')['reading_score'].mean()
eleventh_grade_reading = school_data_complete.loc[school_data_complete['grade'] == '11th'].groupby('school_name')['reading_score'].mean()
twelfth_grade_reading = school_data_complete.loc[school_data_complete['grade'] == '12th'].groupby('school_name')['reading_score'].mean()

In [622]:
#Creating df for reading scores by grade and school
reading_by_grade = pd.DataFrame({'9th': ninth_grade_reading,
                                 '10th': tenth_grade_reading,
                                 '11th': eleventh_grade_reading,
                                 '12th': twelfth_grade_reading})

In [623]:
reading_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [649]:
scores_spending = merge_df

In [650]:
scores_spending = scores_spending.rename(columns={'Per Student Budget': 'Per_Student_Budget'})

In [651]:
#Making Per Student Budget a float from object
scores_spending['Per_Student_Budget'] = scores_spending.Per_Student_Budget.str.replace('$', '').astype(float)

In [652]:
#Creating bins & labels for Spending Ranges (Per Student)
bins = [0, 584, 629, 644, 675]
budget_buckets = ['<$584', '$585-629', '$630-644', '$645-675']

In [653]:
#Placing Per Student Budget into bins
scores_spending['Spending Ranges (Per Student)'] = pd.cut(scores_spending['Per_Student_Budget'], bins, labels=budget_buckets)

In [654]:
scores_by_spending = scores_spending.groupby(['Spending Ranges (Per Student)'])

In [655]:
mean_math_score = scores_by_spending['Average Math Score'].mean()
mean_reading_score = scores_by_spending['Average Reading Score'].mean()
mean_percent_math = scores_by_spending['% Passing Math'].mean()
mean_percent_reading = scores_by_spending['% Passing Reading'].mean()
overall_percent_pass = scores_by_spending['% Overall Passing'].mean()

In [656]:
scores_by_spending = pd.DataFrame({"Average Math Score":mean_math_score,
                                   "Average Reading Score":mean_reading_score,
                                   "% Passing Math":mean_percent_math,
                                   "% Passing Reading":mean_percent_reading,
                                   "% Overall Passing Rate":overall_percent_pass
                                   })

In [657]:
#Formating numbers to 2 decimal places
scores_by_spending['Average Math Score'] = scores_by_spending['Average Math Score'].map("{:.2f}".format)
scores_by_spending['Average Reading Score'] = scores_by_spending['Average Reading Score'].map("{:.2f}".format)
scores_by_spending['% Passing Math'] = scores_by_spending['% Passing Math'].map("{:.2f}".format)
scores_by_spending['% Passing Reading'] = scores_by_spending['% Passing Reading'].map("{:.2f}".format)
scores_by_spending['% Overall Passing Rate'] = scores_by_spending['% Overall Passing Rate'].map("{:.2f}".format)

In [658]:
scores_by_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$584,83.46,83.93,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.52,81.62,73.48,84.39,62.86
$645-675,77.00,81.03,66.16,81.13,53.53


In [659]:
scores_size = merge_df

In [660]:
#Creating bins & labels for School Size
bins = [0, 999, 2000, 5000]
size_buckets = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']

In [661]:
#Placing Total Student into bins
scores_size['School Size'] = pd.cut(scores_size['Total Students'], bins, labels=size_buckets)

In [662]:
scores_by_size = scores_size.groupby(['School Size'])

In [663]:
mean_math_score = scores_by_size['Average Math Score'].mean()
mean_reading_score = scores_by_size['Average Reading Score'].mean()
mean_percent_math = scores_by_size['% Passing Math'].mean()
mean_percent_reading = scores_by_size['% Passing Reading'].mean()
overall_percent_pass = scores_by_size['% Overall Passing'].mean()

In [664]:
scores_by_size = pd.DataFrame({"Average Math Score":mean_math_score,
                               "Average Reading Score":mean_reading_score,
                               "% Passing Math":mean_percent_math,
                               "% Passing Reading":mean_percent_reading,
                               "% Overall Passing Rate":overall_percent_pass
                               })

In [665]:
#Formating numbers to 2 decimal places
scores_by_size['Average Math Score'] = scores_by_size['Average Math Score'].map("{:.2f}".format)
scores_by_size['Average Reading Score'] = scores_by_size['Average Reading Score'].map("{:.2f}".format)
scores_by_size['% Passing Math'] = scores_by_size['% Passing Math'].map("{:.2f}".format)
scores_by_size['% Passing Reading'] = scores_by_size['% Passing Reading'].map("{:.2f}".format)
scores_by_size['% Overall Passing Rate'] = scores_by_size['% Overall Passing Rate'].map("{:.2f}".format)

In [666]:
scores_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (1000-2000),83.37,83.86,93.60,96.79,90.62
Large (2000-5000),77.75,81.34,69.96,82.77,58.29


In [667]:
scores_type = merge_df

In [670]:
#Creating a groupby df on School Type
scores_by_type = scores_type.groupby(['School Type'])

In [671]:
mean_math_score = scores_by_type['Average Math Score'].mean()
mean_reading_score = scores_by_type['Average Reading Score'].mean()
mean_percent_math = scores_by_type['% Passing Math'].mean()
mean_percent_reading = scores_by_type['% Passing Reading'].mean()
overall_percent_pass = scores_by_type['% Overall Passing'].mean()

In [672]:
scores_by_type = pd.DataFrame({"Average Math Score":mean_math_score,
                               "Average Reading Score":mean_reading_score,
                               "% Passing Math":mean_percent_math,
                               "% Passing Reading":mean_percent_reading,
                               "% Overall Passing Rate":overall_percent_pass
                               })

In [673]:
#Formating numbers to 2 decimal places
scores_by_type['Average Math Score'] = scores_by_type['Average Math Score'].map("{:.2f}".format)
scores_by_type['Average Reading Score'] = scores_by_type['Average Reading Score'].map("{:.2f}".format)
scores_by_type['% Passing Math'] = scores_by_type['% Passing Math'].map("{:.2f}".format)
scores_by_type['% Passing Reading'] = scores_by_type['% Passing Reading'].map("{:.2f}".format)
scores_by_type['% Overall Passing Rate'] = scores_by_type['% Overall Passing Rate'].map("{:.2f}".format)

In [674]:
scores_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
